In [ ]:
# file-handling
import os 

# user status updates
import time
from time import gmtime, strftime
from tqdm import tqdm
from IPython.display import display, Latex
from datetime import datetime

# the holy trinity of python
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd

# scipy
import scipy
from scipy import sparse, linalg, fft
from scipy.linalg import expm, sinm, cosm
import scipy.integrate as integrate
from scipy.integrate import quad

# parallelization, memory management
from joblib import Parallel, delayed
from numba import jit, njit, prange
import copy

# itertools
import itertools

############ Macros ###############
np.random.seed(0)

# Physical constants
K = 6 # number of fermionic modes
J = 10 # ~"energy scale"
Q_COUPLING = 3 # order of coupling, don't want to use the letter 'Q' because that denotes the supercharge
N = 2*K # number of fermions
N_DIM = 2**N # Hilbert space dimension <-- Important: Hilbert space dimension is now 2^N instead of 2^(N/2). 

N_SAMPLES = 100 # number of samples to generate
N_JOBS = 20 # number of jobs to run in parallel

# Whether we want to check algebra
CHECK_ALGEBRA = False

# Set up directories
N2_SUSY_DIR = os.path.join("Excel", "N2_SUSY_SYK")
now = strftime("%Y-%m-%d %H:%M:%S", gmtime()).replace("-","_").replace(" ", "_").replace(":","_")
RESULT_DIR = os.path.join(N2_SUSY_DIR, "Results", now)
os.makedirs(RESULT_DIR, exist_ok=True)


# 1. Fermion operators

We've compartmentalized the whole thing (computing fermions, their daggers, pairwise and triple inner-products) to it's own Python function. 

In [ ]:
from N2_SUSY_fermions import make_fermions
psi_dict = make_fermions(N, check_algebra=CHECK_ALGEBRA, n_jobs=N_JOBS)

psi_all = psi_dict["psi_all"]
psi_dagger_all = psi_dict["psi_dagger_all"]
psi_psi_all = psi_dict["psi_psi_all"]
psi_dagger_psi_all = psi_dict["psi_dagger_psi_all"]
psi_psi_dagger_all = psi_dict["psi_psi_dagger_all"]
psi_dagger_psi_dagger_all = psi_dict["psi_dagger_psi_dagger_all"]
psi_psi_psi_all = psi_dict["psi_psi_psi_all"]

# 2. Random coefficients tensor, $C_{ijk}$

$C_{ijk}$ is antisymmetric, has second moment $\overline{C_{ijk} \bar{C}^{ijk}} = \frac{2J}{N^2}$, is complex, and Gaussian. 

1. To achieve antisymmetry: Start by filling in upper-upper-triangle (i.e. where k>j>i), and leave all other elements as zero. Then, for positive permutations of the indeces, *add* this upper-upper-triangle matrix transposed about those axes; conversely, for negative permutations of the indeces, *subtract* the transpose. 

2. To achieve the prescribed variance: Select the upper-upper-triangular elements from some distribution $C' \sim \mathcal{N}(0, \sigma_{C'})$, such that the zero elements along the diagonals and the equal-magnitude elements along the cross-diagonals combine to give the total variance $\sigma_C^2$. We ultimately find: $\sigma_{C'} = \sigma_C \sqrt{\frac{3}{2(N-1)(N-2)}}$. For a derivation of this, see "Upper_upper_triangle_variance.pdf".

3. To achieve complexity: $C'$, the distribution from which the upper-upper-triangular elements are selected, must be generated as the sum of a real and complex distribution: $C' = X' + iY'$, where $X' \sim \mathcal{N}(0, \sigma_{X'})$, $Y \sim \mathcal{N}(0, \sigma_{Y'})$ and - per the variance-addition rule - $\sigma_{X'}=\sigma_{Y'}=\frac{\sigma_{C'}}{\sqrt{2}}$

4. To achieve Gaussian-ness: $X'$ and $Y'$ must be Gaussian, so that the sum - $C'$ is also Gaussian. Whether the zero-elements along the diagonals ruin the Gaussian-ness of the overall tensor: I'm not sure. One *could* consider the diagonals to be distributed as $\sim \mathcal{N}(0,0)$, but that seems fishy...

In [ ]:
sigma_C = np.sqrt(2*J/(N**2))
sigma_C_prime = sigma_C*np.sqrt(3/(2*(N-1)*(N-2)))

# Generate distribution for X', Y'
N_UPPER = N*(N-1)*(N-2)//6
sigma_XY_prime = sigma_C_prime/np.sqrt(2)
X_prime = np.random.normal(0, sigma_XY_prime, size=(N_UPPER))
Y_prime = np.random.normal(0, sigma_XY_prime, size=(N_UPPER))

# Generate distribution for C'
C_prime = X_prime + 1j*Y_prime

# Initialize upper-upper-triangle array
C_upper_upper = np.zeros(shape = [N for i in range(Q_COUPLING)], dtype=np.complex128)

index = 0
for i in range(N-2):
    for j in range(i+1, N-1):
        for k in range(j+1, N):
            C_upper_upper[i, j, k] = C_prime[index]
            index += 1

To antisymmetrize the tensor, we first need a function to antisymmetrize the tuple of axes/indeces (i,j,k)

In [ ]:
def levi_civita_tensor(dim):   
    arr=np.zeros(tuple([dim for i in range(dim)]), dtype=np.int32)
    for x in itertools.permutations(tuple(range(dim))):
        mat = np.zeros((dim, dim), dtype=np.int32)
        for i, j in zip(range(dim), x):
            mat[i, j] = 1
        arr[x]=int(np.linalg.det(mat))
    return arr

def asym_perm(iterable):
    n_elem = len(iterable)
    if len(set(iterable)) < n_elem:
        return 0 # <-- If there are repeated elements, levi-civita value is identically zero 

    order_0 = tuple(list(range(n_elem)))
    all_orders = list(itertools.permutations(order_0))
    all_permutations = [tuple([iterable[i] for i in order]) for order in all_orders]

    lc_tensor = levi_civita_tensor(n_elem)
    out = {}
    for i in range(len(all_orders)):
        order_i = all_orders[i]
        multiplier = lc_tensor[order_i]

        permutation_i = all_permutations[i]
        out[permutation_i] = multiplier

    return out

Now we can antisymmetrize and form the full tensor $C$

In [ ]:
axes_og = (0,1,2)
axes_asym_perms = asym_perm(axes_og)

C = np.zeros(shape=[N for i in range(Q_COUPLING)], dtype=np.complex128)
for axes, multiplier in axes_asym_perms.items():
    addendum = multiplier*np.transpose(C_upper_upper, axes)
    C += addendum

### 2.a Check antisymmetry

In [ ]:
antisymmetric = True
errors = []
for i in range(N):
    for j in range(N):
        for k in range(N):
            C_ijk = C[i,j,k]

            if i==j:
                if C_ijk != 0:
                    antisymmetric=False
                    errors.append(1)

            if i==k:
                if C_ijk != 0:
                    antisymmetric=False
                    errors.append(2)

            if j==k:
                if C_ijk != 0:
                    antisymmetric=False
                    errors.append(3)

            if C_ijk != - C[i,k,j]:
                antisymmetric = False
                errors.append(4)

            if C_ijk != -C[j,i,k]:
                antisymmetric = False
                errors.append(5)

            if C_ijk != C[j,k,i]:
                antisymmetric = False
                errors.append(6)

            if C_ijk != -C[k,j,i]:
                antisymmetric = False
                errors.append(7)

            if C_ijk != C[k,i,j]:
                antisymmetric = False
                errors.append(8)


print(f"antisymmetric: {antisymmetric}")
print(f"errors: {errors}")

### 2.b. Confirm variance

$\overline{C_{ijk} \bar{C}^{ijk}} = \frac{2J}{N^2}$

In [ ]:
declared_variance = sigma_C**2
C_bar = np.conjugate(C)
computed_variance = np.var(C)#np.mean(np.mean(np.mean(C*C_bar)))

print(f"Declared variance: {declared_variance}, Computed variance: {computed_variance}")
print(f"Abs diff: {np.abs(computed_variance-declared_variance)}")
print(f"Percent abs diff: {100*np.abs(computed_variance-declared_variance)/declared_variance:.2f}%")

## 3. Supercharge, $Q$

Compute it

In [ ]:
Q = sparse.csr_array(np.zeros((N_DIM, N_DIM), dtype=np.complex128))
tic = time.time()
for i_index in range(N-2):
    i_label = i_index+1

    for j_index in range(i_index+1, N-1):
        j_label = j_index+1

        for k_index in range(j_index+1, N):
            k_label = k_index+1

            C_ijk = C[i_index, j_index, k_index]
            psi_psi_psi_ijk = psi_psi_psi_all[(i_label, j_label, k_label)]
            Q += C_ijk*psi_psi_psi_ijk
        
    if (i_index==0):
        duration = time.time() - tic
        n_jobs = N-Q_COUPLING
        exp_dur = duration*n_jobs

Q *= 1j
Q_bar = np.transpose(np.conjugate(Q))

print(f"Q hermitian: {linalg.ishermitian(Q.toarray())}")
print(f"Q_bar hermitian: {linalg.ishermitian(Q_bar.toarray())}")

Confirm $Q^2=\bar{Q}^2=0$

In [ ]:
Q2 = (Q@Q).toarray()
display(Latex(f"$Q^2=0$: {np.allclose(Q2, np.zeros(Q2.shape))}"))

Q_bar_2 = (Q_bar@Q_bar).toarray()
display(Latex(f"$Qbar^2=0$: {np.allclose(Q2, np.zeros(Q2.shape))}"))

Confirm equation (5.2.a):

$\{Q, \psi^i\}=0$

In [ ]:
def anticommutator(a,b):
    return a@b+b@a

for i_index in range(N):
    i_label = i_index+1
    
    ac_Q_i = anticommutator(Q, psi_all[i_label]).toarray()
    print(np.allclose(ac_Q_i, np.zeros(ac_Q_i.shape)))

    abs_diff = np.sum(np.sum(np.abs(ac_Q_i)))
    print(f"abs_diff: {abs_diff}\n")

Confirm equation (5.2.b):

$\{Q, \bar{\psi_i}\} = \bar{b^i} = i \sum_{1\le j<k\le N} C_{ijk} \psi^j \psi^k$

In [ ]:
for i_index in range(N):
    i_label = i_index+1
    ac_Q_idagger = anticommutator(Q, psi_dagger_all[i_label]).toarray()

    rhs = np.zeros(Q.shape)
    for j_index in range(N-1):
        j_label = j_index+1
        for k_index in range(j_index+1, N):
            k_label = k_index+1

            C_ijk = C[i_index, j_index, k_index]
            psi_psi_jk = psi_psi_all[(j_label, k_label)]
            rhs += C_ijk*psi_psi_jk

    rhs *= 1j
    print(np.allclose(ac_Q_idagger, rhs, atol=1e-10, rtol=1e-6))

    abs_diff = np.sum(np.sum(np.abs(ac_Q_idagger, rhs)))
    print(f"abs_diff: {abs_diff}\n")

## 4. Hamiltonian

In [ ]:
def anticommutator(A,B):
    return A@B+B@A

Q_bar = np.transpose(np.conjugate(Q))
H = anticommutator(Q, Q_bar)

print(f"H hermitian: {linalg.ishermitian(H.toarray())}")

In [ ]:
iv = np.linalg.eigvalsh(H.toarray())
iv = np.sort(iv)

plt.figure()
plt.hist(iv, bins=30)
plt.xlabel("Eigenvalue")
plt.ylabel("Frequency")
plt.title(r"$N=2$ Supersymmetric SYK Model, "+f"N={N}, J={J}, Q={Q_COUPLING}")
plt.savefig(os.path.join(RESULT_DIR, "iv_hist.png"))

Important question: How small is small-enough to consider it "exactly 0"?

## Save everything

In [ ]:
# Save eigenvalues
np.save(os.path.join(RESULT_DIR, "iv.npy"), iv)
iv_df = pd.DataFrame({"eigenvalue":iv})
iv_df.to_csv(os.path.join(RESULT_DIR,"iv.csv"), index=False)

In [ ]:
# Save Hamiltonian
np.save(os.path.join(RESULT_DIR, "H.npy"), H)
H_df = pd.DataFrame(H.toarray())
H_df.to_csv(os.path.join(RESULT_DIR,"H.csv"), index=False)

In [ ]:
# Save Q and Q_bar
np.save(os.path.join(RESULT_DIR, "Q.npy"), Q)
Q_df = pd.DataFrame(Q.toarray())
Q_df.to_csv(os.path.join(RESULT_DIR, "Q.csv"), index=False)

np.save(os.path.join(RESULT_DIR, "Q_bar.npy"), Q_bar)
Q_bar_df = pd.DataFrame(Q_bar.toarray())
Q_bar_df.to_csv(os.path.join(RESULT_DIR, "Q_bar.csv"), index=False)

In [ ]:
# Save C_prime, C
np.save(os.path.join(RESULT_DIR, "C_prime.npy"), C_prime)
C_prime_df = pd.DataFrame(C_prime)
C_prime_df.to_csv(os.path.join(RESULT_DIR, "C_prime.csv"), index=False)

np.save(os.path.join(RESULT_DIR, "C.npy"), C)
C_dir = os.path.join(RESULT_DIR, "C")
os.makedirs(C_dir, exist_ok=True)
for i in range(N):
    C_i = C[i]
    C_i_dict_temp = {f"C_ij{k+1}": C_i[:,k] for k in range(N)}
    C_i_dict = {"j":list(range(1,N+1))}
    C_i_dict.update(C_i_dict_temp)
    C_i_df = pd.DataFrame(C_i_dict)
    C_i_df.set_index("j", inplace=True)
    C_i_df.to_csv(os.path.join(C_dir, f"C_{i+1}jk.csv"))

In [ ]:
# Save about.txt, explaining the parameters
about_txt = f"N={N} \nJ={J} \nQ_COUPLING={Q_COUPLING}"
with open(os.path.join(RESULT_DIR, "about.txt"), "w") as f:
    f.write(about_txt)